# 🏗️ SiteLenz: Building Defect Classification
## Load & Use Your Trained ViT Model

**Quick Start:**
1. Place `vit_weights.pth` in the `models/` folder
2. Run all cells
3. Use for predictions

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import timm
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import os

In [ ]:
# Configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

MODEL_PATH = 'models/vit_weights.pth'
DATA_DIR = 'data'
NUM_CLASSES = 7
CLASS_NAMES = ['algae', 'major_crack', 'minor_crack', 'peeling', 'plain', 'spalling', 'stain']

## 📦 Load Model

In [ ]:
# Load Vision Transformer model
model = timm.create_model('vit_base_patch16_224', pretrained=False, num_classes=NUM_CLASSES)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model = model.to(device)
model.eval()
print("✅ Model loaded successfully!")

## 🔍 Single Image Prediction

In [ ]:
def predict_image(image_path, model, class_names, device):
    """Predict defect class for a single image"""
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)
    
    with torch.no_grad():
        outputs = model(img_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        confidence, predicted = torch.max(probabilities, 1)
    
    # Display result
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Prediction: {class_names[predicted.item()]}\nConfidence: {confidence.item()*100:.2f}%", 
              fontsize=14, fontweight='bold')
    plt.show()
    
    # Show all probabilities
    print("\n📊 Class Probabilities:")
    for i, prob in enumerate(probabilities[0]):
        print(f"{class_names[i]:15s}: {prob.item()*100:5.2f}%")
    
    return class_names[predicted.item()], confidence.item()

In [ ]:
# Example: Predict a single image
# Replace with your image path
image_path = 'sample images/class_images/Algae/algae (1).jpg'

if os.path.exists(image_path):
    prediction, confidence = predict_image(image_path, model, CLASS_NAMES, device)
else:
    print(f"⚠️ Image not found: {image_path}")
    print("Update the path to test your own image")

## 📈 Evaluate on Test Dataset

In [ ]:
# Load test dataset
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, 'test'), test_transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)
print(f"Test dataset: {len(test_dataset)} images")

In [ ]:
# Evaluate model
def evaluate_model(model, test_loader, device):
    """Calculate accuracy and collect predictions"""
    model.eval()
    all_preds = []
    all_labels = []
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = 100 * correct / total
    return accuracy, np.array(all_preds), np.array(all_labels)

accuracy, predictions, true_labels = evaluate_model(model, test_loader, device)
print(f"\n🎯 Test Accuracy: {accuracy:.2f}%")

## 📊 Confusion Matrix & Classification Report

In [ ]:
# Generate confusion matrix
cm = confusion_matrix(true_labels, predictions)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
plt.title('Confusion Matrix - Vision Transformer', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

# Classification report
print("\n📋 Classification Report:")
print(classification_report(true_labels, predictions, target_names=CLASS_NAMES, digits=4))

## 💾 Batch Prediction (Optional)

In [ ]:
# Predict on multiple images in a folder
def predict_folder(folder_path, model, class_names, device, max_images=10):
    """Predict defects for all images in a folder"""
    results = []
    
    for img_name in os.listdir(folder_path)[:max_images]:
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, img_name)
            try:
                pred, conf = predict_image(img_path, model, class_names, device)
                results.append({'image': img_name, 'prediction': pred, 'confidence': conf})
            except Exception as e:
                print(f"Error processing {img_name}: {e}")
    
    return results

# Example: Predict on a folder (uncomment to use)
# results = predict_folder('sample images/class_images/Algae/', model, CLASS_NAMES, device)
# for r in results:
#     print(f"{r['image']:30s} → {r['prediction']:15s} ({r['confidence']*100:.1f}%)")